<a href="https://colab.research.google.com/github/deartc/FitBitLit/blob/main/FitBitdataColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries


In [12]:
import fitbit
# make sure gather_keys_oauth2.py is in the same directory. 
# you also need to install CherryPy using 'pip install cherrypy' on your virtual environment
import gather_keys_oauth2 as Oauth2
import pandas as pd
import datetime

ModuleNotFoundError: ignored

In [9]:
Update authentication

SyntaxError: ignored

In [10]:
# you need to use your personal CLIENT_ID and CLIENT_SECRET
CLIENT_ID='237YXD'
CLIENT_SECRET='d0d3392191b3ffd6439663b1d4bcfe27'

Get tokens and establish connection with Fitbit API

In [11]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()

ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])

auth2_client=fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

NameError: ignored

pull data

In [ ]:
# here you need to adjust the start date, depending on when you started Fitbit data collection
startTime = datetime.datetime(year = 2022, month = 1, day = 24)
today = datetime.datetime.today().date()
endTime = today - datetime.timedelta(days=1)

In [ ]:
date_list = []
df_list = []

allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    date_list.append(oneDate)
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')
    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    df_list.append(df)

final_df_list = []

for date, df in zip(date_list, df_list):
    if len(df) == 0:
        continue
    df.loc[:, 'date'] = pd.to_datetime(date)
    final_df_list.append(df)

final_df = pd.concat(final_df_list, axis = 0)

In [ ]:
# this adds more granularity to data (date and time)
hoursDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.hour.apply(lambda x: datetime.timedelta(hours = x))
minutesDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.minute.apply(lambda x: datetime.timedelta(minutes = x))
secondsDelta = pd.to_datetime(final_df.loc[:, 'time']).dt.second.apply(lambda x: datetime.timedelta(seconds = x))
final_df['date_and_time'] = final_df['date'] + hoursDelta + minutesDelta + secondsDelta
# optional: generate a csv export

In [ ]:
# optional: generate a csv export
filename = 'export_all_heart_data_' + str(today)
final_df.to_csv(filename + '.csv', index = False)

visualize'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

final_df.plot('date_and_time', 'value', label='heart rate', figsize=(14, 7), title="Evolution of heart rate

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

times = pd.to_datetime(final_df['date_and_time'])
final_df.groupby([times.dt.hour]).value.mean().plot(ax = ax, label='avg')
final_df.groupby([times.dt.hour]).value.min().plot(ax = ax, label='min')
final_df.groupby([times.dt.hour]).value.max().plot(ax = ax, label='max')
plt.legend(loc="upper left")

ax.grid(True, axis = 'both', zorder = 0, linestyle = ':', color = 'k')
ax.tick_params(axis = 'both', rotation = 45, labelsize = 12)

ax.set_xlabel('Hour of the day', fontsize = 14)
ax.set_ylabel('Heart Rate', fontsize = 14)

plt.xticks(range(0, 24))
ax.set_xlim(0, 23)

title = "Min/Avg/Max heart rate per hour across the day from {} to {}".format(startTime.date(), endTime)
ax.set_title(title, fontdict={'fontsize': 14, 'fontweight': 'medium'})

ax.set()
fig.tight_layout()

# to save the graph as an img in the end
fig.savefig('heartRateGraphAvgByMin.png', format = 'png', dpi = 300)